In [107]:
import pandas as pd
import ast
import json
import re
#from difflib import SequenceMatcher
#from geopy.distance import geodesic
#import numpy as np
import numpy as np
from urllib.parse import urlparse
import time

In [ ]:
# Import sample data
sample_df = pd.read_csv('../input_data/processed_samples.csv')
sample_df

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state,address_postcode,base_address_street,base_address_city,base_address_state,base_address_postcode,address_string,base_address_string,categories_list,base_categories_list
0,0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': ""Goin' Postal Jacksonville""}","{'primary': 'shipping_center', 'alternate': ['...",0.996261,['http://www.goinpostaljacksonville.com/'],['https://www.facebook.com/463273470392736'],NaN,...,FL,32256-2892,7643 Gate Pkwy Ste 104,Jacksonville,FL,32256,"7643 Gate Pkwy, Jacksonville, FL, 32256-2892","7643 Gate Pkwy Ste 104, Jacksonville, FL, 32256","['shipping_center', 'freight_and_cargo_service...","['vehicle_shipping', 'courier_and_delivery_ser..."
1,1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,['http://valleytransca.com/'],['https://www.facebook.com/107150256087765'],NaN,...,CA,92021-5023,1158 N 2nd St,El Cajon,CA,92021,"1158 N 2nd St, El Cajon, CA, 92021-5023","1158 N 2nd St, El Cajon, CA, 92021","['automotive_repair', 'automotive', 'professio...","['transmission_repair', 'automotive']"
2,3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,['https://stores.redwingshoes.com/roswell-ga'],NaN,NaN,...,GA,30076,875 Mansell Rd Ste B1,Roswell,GA,30076,"875 Mansell Rd B1, Roswell, GA, 30076","875 Mansell Rd Ste B1, Roswell, GA, 30076","['shoe_store', 'online_shop', 'retail']","['shoe_store', 'online_shop', 'retail']"
3,5,1125899912143279,1407374890190043,"[{'dataset': 'Microsoft', 'record_id': '112589...",{'primary': 'Newport Institute'},"{'primary': 'doctor', 'alternate': ['health_an...",0.770000,['http://www.newportinstitute.com/?utm_source=...,NaN,NaN,...,CA,92590,43821 Shady Creek,Temecula,CA,92590,"43821 Shady Crk, Temecula, CA, 92590","43821 Shady Creek, Temecula, CA, 92590","['doctor', 'health_and_medical']","['doctor', 'health_and_medical']"
4,6,08f2aa879200ab960398d7c665470cf4,562949954974969,"[{'property': '', 'dataset': 'msft', 'record_i...",{'primary': 'Comfort Inn Alexandria West - Lan...,{'primary': 'hotel'},0.770000,['https://www.choicehotels.com/virginia/alexan...,NaN,NaN,...,VA,22312,6254 Duke St,Alexandria,VA,22312,"6254 Duke St, Alexandria, VA, 22312","6254 Duke St, Alexandria, VA, 22312",['hotel'],['hotel']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,1976,08f44c10dd244c09032cb0144487c85b,dba914f3e9e247fb64f813c7,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Edward Jones- Financial Advisor:H...,"{'primary': 'financial_advising', 'alternate':...",0.231548,['http://www.edwardjones.com/'],['https://www.facebook.com/536502823203299'],NaN,...,GA,30078-3071,2795 Main St W Ste 24B,Snellville,GA,30078,"2795 Main St W, Snellville, GA, 30078-3071","2795 Main St W Ste 24B, Snellville, GA, 30078","['financial_advising', 'financial_service']",['Business and Professional Services > Financi...
956,1979,08f28d542c8d230903943451a3bc73cf,4bc4c2c3abf49521d686c593,"[{'property': '', 'dataset': 'Microsoft', 'rec...",{'primary': 'Starbucks'},"{'primary': 'coffee_shop', 'alternate': ['rest...",0.770000,['https://www.starbucks.com/store-locator/stor...,NaN,NaN,...,WA,98101,400 Pine St,Seattle,WA,98101,"1400 6th Ave, Seattle, WA, 98101","400 Pine St, Seattle, WA, 98101","['coffee_shop', 'restaurant']","['Dining and Drinking > Cafe, Coffee, and Tea ..."
957,1981,08f264626394cd4b030ac3f573c76061,4ca4f68af47ea1434edd5221,"[{'property': '', 'dataset': 'Microsoft', 'rec...",{'primary': 'Bank of America ATM'},"{'primary': 'online_shop', 'alternate': ['bank...",0.770000,['https://locators.bankofamerica.com/mo/richmo..

In [109]:
# Check current types
sample_df.map(type)

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state,address_postcode,base_address_street,base_address_city,base_address_state,base_address_postcode,address_string,base_address_string,categories_list,base_categories_list
0,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
956,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
957,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
958,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [110]:
# Assigns data types to attributes
def parse_category(x):
    if isinstance(x, (list, dict)) or pd.isna(x): return x
    try: return ast.literal_eval(x)
    except: return None

for c in ["sources", "names", "addresses","base_names","base_addresses","base_categories", "categories", "websites", "brand"]:
    if c in sample_df: sample_df[c] = sample_df[c].apply(parse_category)

sample_df.map(type)

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state,address_postcode,base_address_street,base_address_city,base_address_state,base_address_postcode,address_string,base_address_string,categories_list,base_categories_list
0,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
956,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
957,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
958,<class 'int'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'float'>,<class 'float'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [111]:
sample_df

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state,address_postcode,base_address_street,base_address_city,base_address_state,base_address_postcode,address_string,base_address_string,categories_list,base_categories_list
0,0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],['https://www.facebook.com/463273470392736'],NaN,...,FL,32256-2892,7643 Gate Pkwy Ste 104,Jacksonville,FL,32256,"7643 Gate Pkwy, Jacksonville, FL, 32256-2892","7643 Gate Pkwy Ste 104, Jacksonville, FL, 32256","['shipping_center', 'freight_and_cargo_service...","['vehicle_shipping', 'courier_and_delivery_ser..."
1,1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],['https://www.facebook.com/107150256087765'],NaN,...,CA,92021-5023,1158 N 2nd St,El Cajon,CA,92021,"1158 N 2nd St, El Cajon, CA, 92021-5023","1158 N 2nd St, El Cajon, CA, 92021","['automotive_repair', 'automotive', 'professio...","['transmission_repair', 'automotive']"
2,3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],NaN,NaN,...,GA,30076,875 Mansell Rd Ste B1,Roswell,GA,30076,"875 Mansell Rd B1, Roswell, GA, 30076","875 Mansell Rd Ste B1, Roswell, GA, 30076","['shoe_store', 'online_shop', 'retail']","['shoe_store', 'online_shop', 'retail']"
3,5,1125899912143279,1407374890190043,"[{'dataset': 'Microsoft', 'record_id': '112589...",{'primary': 'Newport Institute'},"{'primary': 'doctor', 'alternate': ['health_an...",0.770000,[http://www.newportinstitute.com/?utm_source=s...,NaN,NaN,...,CA,92590,43821 Shady Creek,Temecula,CA,92590,"43821 Shady Crk, Temecula, CA, 92590","43821 Shady Creek, Temecula, CA, 92590","['doctor', 'health_and_medical']","['doctor', 'health_and_medical']"
4,6,08f2aa879200ab960398d7c665470cf4,562949954974969,"[{'property': '', 'dataset': 'msft', 'record_i...",{'primary': 'Comfort Inn Alexandria West - Lan...,{'primary': 'hotel'},0.770000,[https://www.choicehotels.com/virginia/alexand...,NaN,NaN,...,VA,22312,6254 Duke St,Alexandria,VA,22312,"6254 Duke St, Alexandria, VA, 22312","6254 Duke St, Alexandria, VA, 22312",['hotel'],['hotel']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,1976,08f44c10dd244c09032cb0144487c85b,dba914f3e9e247fb64f813c7,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Edward Jones- Financial Advisor:H...,"{'primary': 'financial_advising', 'alternate':...",0.231548,[http://www.edwardjones.com/],['https://www.facebook.com/536502823203299'],NaN,...,GA,30078-3071,2795 Main St W Ste 24B,Snellville,GA,30078,"2795 Main St W, Snellville, GA, 30078-3071","2795 Main St W Ste 24B, Snellville, GA, 30078","['financial_advising', 'financial_service']",['Business and Professional Services > Financi...
956,1979,08f28d542c8d230903943451a3bc73cf,4bc4c2c3abf49521d686c593,"[{'property': '', 'dataset': 'Microsoft', 'rec...",{'primary': 'Starbucks'},"{'primary': 'coffee_shop', 'alternate': ['rest...",0.770000,[https://www.starbucks.com/store-locator/store...,NaN,NaN,...,WA,98101,400 Pine St,Seattle,WA,98101,"1400 6th Ave, Seattle, WA, 98101","400 Pine St, Seattle, WA, 98101","['coffee_shop', 'restaurant']","['Dining and Drinking > Cafe, Coffee, and Tea ..."
957,1981,08f264626394cd4b030ac3f573c76061,4ca4f68af47ea1434edd5221,"[{'property': '', 'dataset': 'Microsoft', 'rec...",{'primary': 'Bank of America ATM'},"{'primary': 'online_shop', 'alternate': ['bank...",0.770000,[https://locators.bankofamerica.com/mo/richmon...,NaN,NaN,..

# Attribute Conflation Labeling Strategy

The `sample_df` dataset contains pairs of places where each row represents the same physical place from two different data sources. The "base_" prefixed columns represent Place A (source A) and the non-prefixed columns represent Place B (source B). 

Goal: Create a labeling system to determine which attributes are more reliable for building a universal places database.

## Strategy Overview

Two labeling approaches:

1. **Per-Attribute Binary Labels**: For each attribute type, determine if Place A or Place B has the better/more reliable value
2. **Overall Quality Score**: An aggregate score indicating which place (A or B) has better overall data quality

## Key Attributes to Label:
- Names
- Addresses 
- Categories
- Websites
- Socials
- Emails
- Phones
- Brand

In [112]:
# Labeling Strategy Implementation

class AttributeQualityEvaluator:
    """
    Evaluates quality of attributes for place conflation labeling
    """
    
    def __init__(self):
        self.attribute_weights = {
            'names': 0.25,
            'addresses': 0.20, 
            'categories': 0.15,
            'websites': 0.10,
            'phones': 0.10,
            'emails': 0.10,
            'socials': 0.05,
            'brand': 0.05
        }
    
    def evaluate_name_quality(self, name_data):
        """Evaluate name quality based on completeness and structure"""
        if not name_data or pd.isna(name_data):
            return 0.0
            
        if isinstance(name_data, dict):
            score = 0.0
            # Primary name exists
            if name_data.get('primary'):
                score += 0.6
                # Bonus for reasonable length
                if 5 <= len(name_data['primary']) <= 100:
                    score += 0.2
            # Alternative names exist
            if name_data.get('alternatives') and len(name_data['alternatives']) > 0:
                score += 0.2
            return min(score, 1.0)
        
        # If it's a string, basic scoring
        if isinstance(name_data, str):
            if 5 <= len(name_data) <= 100:
                return 0.8
            return 0.4
        
        return 0.0
    
    def evaluate_address_quality(self, address_data):
        """Evaluate address quality based on completeness and structure"""
        if not address_data or pd.isna(address_data):
            return 0.0
            
        if isinstance(address_data, list) and len(address_data) > 0:
            addr = address_data[0]
            score = 0.0
            
            # Street address component
            if addr.get('street'):
                score += 0.3
            
            # City component
            if addr.get('locality') or addr.get('city'):
                score += 0.25
            
            # Postal code component
            if addr.get('postcode'):
                pc = str(addr.get('postcode'))
                # Bonus for valid format
                if re.match(r'^\d{5}(-\d{4})?$', pc):  # US ZIP
                    score += 0.25
                else:
                    score += 0.15
            
            # Country component
            if addr.get('country'):
                score += 0.1
            
            # State/region component
            if addr.get('region') or addr.get('state'):
                score += 0.1
            
            return min(score, 1.0)
        
        return 0.0
    
    def evaluate_categories_quality(self, categories_data):
        """Evaluate categories quality based on number and specificity"""
        if not categories_data or pd.isna(categories_data):
            return 0.0
        
        if isinstance(categories_data, list):
            if len(categories_data) == 0:
                return 0.0
            
            score = 0.0
            # Base score for having categories
            score += 0.4
            
            # Bonus for multiple categories (up to a point)
            num_cats = min(len(categories_data), 5)
            score += (num_cats - 1) * 0.1
            
            # Bonus for specific categories (longer names often more specific)
            avg_length = np.mean([len(str(cat)) for cat in categories_data])
            if avg_length > 10:
                score += 0.2
            
            return min(score, 1.0)
        
        return 0.0
    
    def evaluate_website_quality(self, website_data):
        """Evaluate website quality based on validity and completeness"""
        if not website_data or pd.isna(website_data):
            return 0.0
        
        if isinstance(website_data, list):
            valid_urls = 0
            for url in website_data:
                if self.is_valid_url(url):
                    valid_urls += 1
            
            if valid_urls > 0:
                return min(0.5 + (valid_urls - 1) * 0.2, 1.0)
        
        return 0.0
    
    def evaluate_contact_quality(self, contact_data, contact_type='phone'):
        """Evaluate phone/email quality"""
        if not contact_data or pd.isna(contact_data):
            return 0.0
        
        if isinstance(contact_data, list):
            valid_contacts = 0
            for contact in contact_data:
                if contact_type == 'phone' and self.is_valid_phone(contact):
                    valid_contacts += 1
                elif contact_type == 'email' and self.is_valid_email(contact):
                    valid_contacts += 1
            
            if valid_contacts > 0:
                return min(0.6 + (valid_contacts - 1) * 0.2, 1.0)
        
        return 0.0
    
    def evaluate_social_quality(self, social_data):
        """Evaluate social media quality"""
        if not social_data or pd.isna(social_data):
            return 0.0
        
        if isinstance(social_data, list) and len(social_data) > 0:
            return min(0.3 + len(social_data) * 0.2, 1.0)
        
        return 0.0
    
    def evaluate_brand_quality(self, brand_data):
        """Evaluate brand quality"""
        if not brand_data or pd.isna(brand_data):
            return 0.0
        
        if isinstance(brand_data, str) and len(brand_data) > 2:
            return 0.8
        
        return 0.0
    
    def is_valid_url(self, url):
        """Check if URL is valid"""
        try:
            result = urlparse(str(url))
            return all([result.scheme, result.netloc])
        except:
            return False
    
    def is_valid_phone(self, phone):
        """Check if phone number looks valid"""
        if not phone:
            return False
        
        # Remove common formatting
        clean_phone = re.sub(r'[^\d+]', '', str(phone))
        # Basic validation - reasonable length
        return 7 <= len(clean_phone) <= 15
    
    def is_valid_email(self, email):
        """Check if email looks valid"""
        if not email:
            return False
        
        email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return bool(re.match(email_pattern, str(email)))

# Initialize evaluator
evaluator = AttributeQualityEvaluator()

In [113]:
def generate_attribute_labels(row):
    """
    Generate per-attribute and overall quality labels for a place pair
    
    Returns:
    - Per-attribute labels: 1 if Place A (base_) is better, 0 if Place B is better
    - Overall label: 1 if Place A is better overall, 0 if Place B is better overall
    - Quality scores for transparency
    """
    
    labels = {}
    place_a_scores = {}
    place_b_scores = {}
    
    # Evaluate each attribute pair
    attribute_pairs = [
        ('names', 'base_names'),
        ('addresses', 'base_addresses'), 
        ('categories', 'base_categories'),
        ('websites', 'base_websites'),
        ('phones', 'base_phones'),
        ('emails', 'base_emails'),
        ('socials', 'base_socials'),
        ('brand', 'base_brand')
    ]
    
    for attr_b, attr_a in attribute_pairs:
        if attr_a in row.index and attr_b in row.index:
            # Get quality scores
            if attr_b == 'names':
                score_a = evaluator.evaluate_name_quality(row[attr_a])
                score_b = evaluator.evaluate_name_quality(row[attr_b])
            elif attr_b == 'addresses':
                score_a = evaluator.evaluate_address_quality(row[attr_a])
                score_b = evaluator.evaluate_address_quality(row[attr_b])
            elif attr_b == 'categories':
                score_a = evaluator.evaluate_categories_quality(row[attr_a])
                score_b = evaluator.evaluate_categories_quality(row[attr_b])
            elif attr_b == 'websites':
                score_a = evaluator.evaluate_website_quality(row[attr_a])
                score_b = evaluator.evaluate_website_quality(row[attr_b])
            elif attr_b in ['phones', 'emails']:
                contact_type = 'phone' if attr_b == 'phones' else 'email'
                score_a = evaluator.evaluate_contact_quality(row[attr_a], contact_type)
                score_b = evaluator.evaluate_contact_quality(row[attr_b], contact_type)
            elif attr_b == 'socials':
                score_a = evaluator.evaluate_social_quality(row[attr_a])
                score_b = evaluator.evaluate_social_quality(row[attr_b])
            elif attr_b == 'brand':
                score_a = evaluator.evaluate_brand_quality(row[attr_a])
                score_b = evaluator.evaluate_brand_quality(row[attr_b])
            else:
                score_a = score_b = 0.0
            
            # Store scores
            place_a_scores[attr_b] = score_a
            place_b_scores[attr_b] = score_b
            
            # Create binary label (1 if A is better, 0 if B is better)
            # If scores are very close (within 0.1), prefer the one with actual data
            if abs(score_a - score_b) < 0.1:
                # Tie-breaking: prefer non-null data
                has_data_a = row[attr_a] is not None and not pd.isna(row[attr_a])
                has_data_b = row[attr_b] is not None and not pd.isna(row[attr_b])
                
                if has_data_a and not has_data_b:
                    labels[f'label_{attr_b}'] = 1
                elif has_data_b and not has_data_a:
                    labels[f'label_{attr_b}'] = 0
                else:
                    # True tie
                    labels[f'label_{attr_b}'] = 2
            else:
                labels[f'label_{attr_b}'] = 1 if score_a > score_b else 0
    
    # Calculate overall weighted score
    total_weight_a = 0
    total_weight_b = 0
    
    for attr_b in place_a_scores.keys():
        if attr_b in evaluator.attribute_weights:
            weight = evaluator.attribute_weights[attr_b]
            total_weight_a += weight * place_a_scores[attr_b]
            total_weight_b += weight * place_b_scores[attr_b]
    
    # Overall label
    labels['label_overall'] = 1 if total_weight_a > total_weight_b else 0
    
    # Add quality scores for transparency
    labels['quality_score_a'] = total_weight_a
    labels['quality_score_b'] = total_weight_b
    labels['quality_difference'] = total_weight_a - total_weight_b
    
    return pd.Series(labels)

# Test the labeling function on first few rows
print("Testing labeling function on first 3 rows:")
test_labels = sample_df.head(3).apply(generate_attribute_labels, axis=1)
print(test_labels)

Testing labeling function on first 3 rows:
   label_names  label_addresses  label_categories  label_websites  \
0          2.0              2.0               2.0             0.0   
1          2.0              2.0               2.0             0.0   
2          2.0              2.0               2.0             0.0   

   label_phones  label_emails  label_socials  label_brand  label_overall  \
0           2.0           2.0            0.0          1.0            0.0   
1           2.0           2.0            0.0          1.0            0.0   
2           2.0           2.0            2.0          1.0            0.0   

   quality_score_a  quality_score_b  quality_difference  
0             0.38             0.39               -0.01  
1             0.38             0.39               -0.01  
2             0.38             0.39               -0.01  


In [114]:
# Apply labeling to entire dataset
print("Generating labels for all 2000 place pairs...")
start_time = time.time()

# Generate labels for all rows
labels_df = sample_df.apply(generate_attribute_labels, axis=1)

# Combine with original data
labeled_sample_df = pd.concat([sample_df, labels_df], axis=1)

end_time = time.time()
print(f"Labeling time: {end_time - start_time:.2f} seconds")

# Display labeling summary
print("\n=== LABELING SUMMARY ===")
print(f"Total place pairs labeled: {len(labeled_sample_df)}")

print("\nPer-attribute label distribution:")
attribute_labels = [col for col in labels_df.columns if col.startswith('label_') and col != 'label_overall']
for label_col in attribute_labels:
    attr_name = label_col.replace('label_', '')
    count_a = (labels_df[label_col] == 1).sum()
    count_b = (labels_df[label_col] == 0).sum()
    print(f"  {attr_name:12s}: Place A (base_) better: {count_a:4d} | Place B better: {count_b:4d}")

print(f"\nOverall quality distribution:")
overall_a = (labels_df['label_overall'] == 1).sum()
overall_b = (labels_df['label_overall'] == 0).sum()
print(f"  Place A (base_) better overall: {overall_a}")
print(f"  Place B better overall: {overall_b}")

print(f"\nQuality score statistics:")
print(f"  Place A mean quality: {labels_df['quality_score_a'].mean():.3f}")
print(f"  Place B mean quality: {labels_df['quality_score_b'].mean():.3f}")
print(f"  Mean quality difference (A-B): {labels_df['quality_difference'].mean():.3f}")
print(f"  Quality difference std: {labels_df['quality_difference'].std():.3f}")

Generating labels for all 2000 place pairs...
Labeling time: 0.39 seconds

=== LABELING SUMMARY ===
Total place pairs labeled: 960

Per-attribute label distribution:
  names       : Place A (base_) better:    1 | Place B better:    5
  addresses   : Place A (base_) better:    7 | Place B better:   12
  categories  : Place A (base_) better:    3 | Place B better:    0
  websites    : Place A (base_) better:    9 | Place B better:  905
  phones      : Place A (base_) better:    6 | Place B better:    1
  emails      : Place A (base_) better:   77 | Place B better:    0
  socials     : Place A (base_) better:    0 | Place B better:  545
  brand       : Place A (base_) better:  934 | Place B better:    3

Overall quality distribution:
  Place A (base_) better overall: 55
  Place B better overall: 905

Quality score statistics:
  Place A mean quality: 0.377
  Place B mean quality: 0.386
  Mean quality difference (A-B): -0.009
  Quality difference std: 0.017
Labeling time: 0.39 seconds

=== 

In [115]:
# Create detailed analysis and examples

# 1. Show examples of clear winners
print("=== EXAMPLES OF CLEAR QUALITY DIFFERENCES ===")

# Find rows with largest quality differences
top_a_indices = labels_df.nlargest(3, 'quality_difference').index
top_b_indices = labels_df.nsmallest(3, 'quality_difference').index

print("\nTop 3 cases where Place A (base_) is clearly better:")
for i, idx in enumerate(top_a_indices):
    row = labeled_sample_df.iloc[idx]
    print(f"\nExample {i+1} (Row {idx}):")
    print(f"  Quality difference: {row['quality_difference']:.3f}")
    print(f"  Place A name: {row['base_names']}")
    print(f"  Place B name: {row['names']}")
    print(f"  Place A address: {row['base_addresses']}")
    print(f"  Place B address: {row['addresses']}")

print("\n\nTop 3 cases where Place B is clearly better:")
for i, idx in enumerate(top_b_indices):
    row = labeled_sample_df.iloc[idx]
    print(f"\nExample {i+1} (Row {idx}):")
    print(f"  Quality difference: {row['quality_difference']:.3f}")
    print(f"  Place A name: {row['base_names']}")
    print(f"  Place B name: {row['names']}")
    print(f"  Place A websites: {row['base_websites']}")
    print(f"  Place B websites: {row['websites']}")

# 2. Analyze attribute correlation
print("\n\n=== ATTRIBUTE CORRELATION ANALYSIS ===")

# Create correlation matrix for binary labels
label_cols = [col for col in labels_df.columns if col.startswith('label_') and col != 'label_overall']
correlation_matrix = labels_df[label_cols].corr()

print("Correlation between attribute labels:")
print(correlation_matrix.round(3))

# 3. Quality score distribution
print(f"\n=== QUALITY SCORE DISTRIBUTION ===")
print(f"Quality difference distribution:")
print(f"  Min: {labels_df['quality_difference'].min():.3f}")
print(f"  25th percentile: {labels_df['quality_difference'].quantile(0.25):.3f}")
print(f"  Median: {labels_df['quality_difference'].median():.3f}")
print(f"  75th percentile: {labels_df['quality_difference'].quantile(0.75):.3f}")
print(f"  Max: {labels_df['quality_difference'].max():.3f}")

# 4. Identify challenging cases (very close quality scores)
close_cases = labeled_sample_df[abs(labeled_sample_df['quality_difference']) < 0.01]
print(f"\nChallenging cases (quality difference < 0.01): {len(close_cases)} out of {len(labeled_sample_df)}")

=== EXAMPLES OF CLEAR QUALITY DIFFERENCES ===

Top 3 cases where Place A (base_) is clearly better:

Example 1 (Row 141):
  Quality difference: 0.110
  Place A name: {'primary': 'Scripps Memorial Hospital La Jolla Outpatient Imaging Pavillion'}
  Place B name: {'primary': 'Scripps Memorial Hospital La Jolla Outpatient Imaging Pavillion'}
  Place A address: [{'freeform': '9838 Genesee Ave', 'locality': 'La Jolla', 'region': 'CA', 'country': 'US', 'postcode': '92037'}]
  Place B address: [{'freeform': '9838 Genesee Ave', 'postcode': '92037', 'country': 'US'}]

Example 2 (Row 721):
  Quality difference: 0.110
  Place A name: {'primary': 'Rocky Hill', 'common': {}, 'rules': []}
  Place B name: {'primary': 'Rocky Hill Mountain'}
  Place A address: [{'freeform': 'Rocky hill drive', 'locality': 'Exeter', 'region': 'CA', 'country': 'US', 'postcode': '93221'}]
  Place B address: [{'freeform': 'Rocky hill drive', 'postcode': '93221', 'country': 'US'}]

Example 3 (Row 857):
  Quality difference: 

In [116]:
def export_labeled_dataset(df, filename='../output_data/weighted_algorithm_labels.csv'):
    """Export the labeled dataset with comprehensive metadata"""
    
    # Select columns to export (include sample_idx from the original CSV)
    # Include base and non_base attributes for reference and future comparison
    base_columns = ['sample_idx', 'id', 'base_id']
    
    # Add all extracted/processed and original attributes for comparison to Yelp dataset
    attribute_columns = [
        # Names - extracted and original
        'name_primary', 'base_name_primary',
        'names', 'base_names',
        # Categories - extracted and original
        'categories_list', 'base_categories_list',
        'categories', 'base_categories',
        # Addresses - extracted and original
        'address_string', 'base_address_string',
        'addresses', 'base_addresses',
        # Individual address components
        'address_street', 'address_city', 'address_state', 'address_postcode',
        'base_address_street', 'base_address_city', 'base_address_state', 'base_address_postcode',
        # Websites
        'websites', 'base_websites',
        # Contact info
        'phones', 'base_phones',
        'emails', 'base_emails',
        # Social media
        'socials', 'base_socials',
        # Brand
        'brand', 'base_brand'
    ]
    
    # Add label and quality columns
    label_quality_columns = [col for col in df.columns if col.startswith('label_') or col.startswith('quality_')]
    
    # Combine all columns
    export_columns = base_columns + attribute_columns + label_quality_columns
    
    # Filter to only include columns that exist in the dataframe
    export_columns = [col for col in export_columns if col in df.columns]
    
    simplified_df = df[export_columns].copy()
    
    simplified_df.to_csv(filename, index=False)
    print(f"✓ Exported {len(simplified_df)} labeled records to {filename}")
    print(f"  Columns exported: {len(export_columns)}")
    print(f"  Includes: identifiers, all base and non_base attributes (extracted & original), labels, and quality scores")
    

export = export_labeled_dataset(labeled_sample_df)


✓ Exported 960 labeled records to ../output_data/weighted_algorithm_labels.csv
  Columns exported: 45
  Includes: identifiers, all base and non_base attributes (extracted & original), labels, and quality scores
